<a href="https://colab.research.google.com/github/josejamilena/386bsd/blob/0.0/rsa_encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This article outlines the basic ideas of encryption and explains and implements the famous RSA encryption scheme.**

It should not be used for production but only demonstration as it uses Python's pseudorandom generator which is not cryptographically secure.

# What is encryption?
**Encryption** is a function or a mapping that takes in your message (plain text) and converts it into garbled text (sequence of numbers) that you can send over servers without anyone being able to figure out what does it mean.



![alt text](https://4.bp.blogspot.com/-6X64g4j2yIo/W4LIHKSfyuI/AAAAAAAABKc/eoO50t7cJC4agNkXSaMAfvK-17eyIytogCLcBGAs/s1600/Symmetric%2Bkey.PNG)

eg. You want to send a message to your friend. You both decide that you will replace all the letters to their position in the alphabet.

Say you want to send "hey have you got weed", you can instead send:

In [0]:
#@title
message = "hey have you got weed"
cipher = [ord(l)-96 for l in message]
print(cipher)

Now your friend receives this list of numbers, looks up in the Alphabet and replaces each number back to the alphabet.

In [0]:
#@title
message_decrypted = [chr(l+96) for l in cipher]
print(''.join(message_decrypted))

He replies back:

In [0]:
#@title
message = "yes"
cipher = [ord(l)-96 for l in message]
print(cipher)



---

# RSA encryption
RSA is a similar function that it takes each character in your message and maps it to a number and sends this number over the server.

This is an asymmetric key algorithm where there is an encryption and decryption key. Since it is slow to implemenet, it is used to send the symmetric keys instead of encrypting the data.


Let's go through some Maths basics first to refresh what we learned in school.

# Greatest Common Divisor

**Euclid's algorithm** for determining the greatest common divisor (GCD).

a is larger than b, divide a by b, divide quotient by remainder, repeat till remainder is 0


In [0]:
#@title
'''
Euclid's algorithm for determining the greatest common divisor
Use iteration to make it faster for larger integers
a is larger than b, divide a by b, divide quotient by remainder, repeat till remainder is 0
'''
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

In [0]:
gcd(5,10)

# Relatively prime or coprimes
Two numbers a and b are relatively primes iff $gcd(a,b) = 1$ which means that they only share $1$ as their common factor.

For example, 14 and 15 are relatively primes

# Congurent numbers

> $a ≡ b  (\text{mod m})$ ---> a is congruent to b modulo m

which implies m divides a-b

> $a - b = m * Factor$

> $b = remainder(a,m)$

# Multliplicative inverse

x and a are multiplicate inverse of each other mod m if

> $a * x ≡ 1 (\text{mod m})$, where x lies in [1, .. , m-1]

which means m divides a * x - 1

> $a*x - 1 = m * Factor$

> $a*x = m * Factor + 1$

> $1 = remainder(ax,m)$

*It can be shown that if $GCD(n,k) =1$, then $k$ has a multiplicative inverse mod n*

In [0]:
#@title
'''
Euclid's extended algorithm for finding the multiplicative inverse of two numbers
ax ≡ 1 (mod m), x lies in [1, .. , m-1]
'''
def multiplicative_inverse(a, m):
    a = a % m; 
    for x in range(1, m) : 
        if ((a * x) % m == 1) : 
            return x 
    return 1

In [0]:
# example test

# Prime number

In order to check if a number N is prime, we start dividing it with numbers from 2 onwards. Since

$N = \sqrt{N} * \sqrt{N} $

the factor can't be greater than $\sqrt{N}$

So we only have to check numbers till then.

In [0]:
'''
Tests to see if a number is prime.
'''
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2): # cool trick here...
        if num % n == 0:
            return False
    return True

In [0]:
# example
is_prime(13441)

# RSA - Rivest–Shamir–Adleman
Published in 1977 in MIT [pdf](https://people.csail.mit.edu/rivest/Rsapaper.pdf)

# Generation of public and private keys

1.   Generate two prime numbers $p$ and $q$

2.   Define $N = p * q$ called a *semiprime* (product of two primes)

3.   Define $ \phi =  (p-1) * (q-1)$ which is Euler Totient function (see Euler's theorem)

4.   Generate encryption key $e$ such that $gcd(e, \phi) = 1 $ where e lies between $1$ to $\phi -1 $

    i.e. e is relatively prime to $\phi$

   **Public key**: $(e, N)$ visible to everyone

5.   Generate decryption key $d$ such that $d$ is multiplicate inverse of $e$ module $\phi$

    $d*e ≡ 1  (mod \phi)$

    **Private key**: $(d, N)$ visible only to sender and your receiver.


In [0]:
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')
    #n = pq
    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime or relatively prime
    e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are coprime
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key de ≡ 1 (mod phi)
    d = multiplicative_inverse(e, phi)
    
    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

In [0]:
#@title
import random
print("RSA Encrypter/ Decrypter")
p = int(input("Enter a prime number (17, 19, 23, etc): "))
q = int(input("Enter another prime number (Not one you entered above): "))

print("Generating your public/private keypairs now . . .")
public, private = generate_keypair(p, q)
print("Your public key is ", public ," and your private key is ", private)

RSA Encrypter/ Decrypter
Enter a prime number (17, 19, 23, etc): 23
Enter another prime number (Not one you entered above): 331
Generating your public/private keypairs now . . .
Your public key is  (3653, 7613)  and your private key is  (4577, 7613)


# Encryption step
Message $m$ -> cipher $m'$


>   $m' = remainder(m^e, N)$



In [0]:
def encrypt(pk, plaintext):
    #Unpack the key into it's components
    key, n = pk
    #Convert each letter in the plaintext to numbers based on the character using rem(m^e,n) i.e. m^e mod n
    cipher = [(ord(char) ** key) % n for char in plaintext]
    #Return the array of bytes
    return cipher

In [0]:
message = input("Enter a message to encrypt with your public key: ")
encrypted_msg = encrypt(public, message)
print("Your encrypted message is: ")
print(''.join(map(lambda x: str(x), encrypted_msg)))

Enter a message to encrypt with your public key: hello
Your encrypted message is: 
1116426458125812111


# Decryption step
cipher $m'$ -> message $m$


>   $m = remainder(m'^d, N)$


In [0]:
def decrypt(pk, ciphertext):
    #Unpack the key into its components
    key, n = pk
    #Generate the plaintext based on the ciphertext and key using rem(m'^d,n)
    plain = [chr((char ** key) % n) for char in ciphertext]
    #Return the array of bytes as a string
    return ''.join(plain)

In [0]:
print("Decrypting message with private key ", public ," . . .")
print("Your message is:")
print(decrypt(private, encrypted_msg))

Decrypting message with private key  (3653, 7613)  . . .
Your message is:
hello


# Euler's theorems and Fermat's Little theorem

Euler totient function $\phi(n)$ is defined as the number of integers in $[1,2,3....,n-1]$ that are relatively prime to $n$



1.   If $p1, p2, ..., pj$ are the prime factors of $n$, then
$\phi(n) = n (1 - \frac{1}{p1}) ..  (1 - \frac{1}{pj}) $

2.   If $n = p * q$ where $p$ and $q$ are prime (n is called semiprime) then it can be show that $\phi(n) = (p-1) (q-1)$

3.   For all $n,k$ s.t. $GCD(n,k)=1$, then it can be shown that $k^{\phi(n)} ≡  1 (\text{mod n)}$

4.   If $n$ is prime s.t. $n = p$, then $k^{p-1} ≡  1 (\text{mod p)}$



# More on RSA

*    RSA on [Wiki](https://en.wikipedia.org/wiki/RSA_(cryptosystem))

*    Code used was modified from [here](https://gist.github.com/JonCooperWorks/5314103)
*    [RSA challenges](https://en.wikipedia.org/wiki/RSA_numbers) How many years does it take to find the prime factors of a xxx-bit semiprimes

*    Shor's algorithm to break RSA using quantum computers (fast) or classical computers (slow) ([youtube](https://www.youtube.com/watch?v=lvTqbM5Dq4Q))



# More on prime numbers
*    **Prime number theorem ([wiki](https://en.wikipedia.org/wiki/Prime_number_theorem#Proof_sketch)):**  It formalizes the intuitive idea that primes become less common as they become larger by precisely quantifying the rate at which this occurs. Specifically $pi(N) \approx \frac{N}{log(N)}$

*    Database of primes upto $10^{12}$ [here](https://primes.utm.edu/nthprime/index.php#nth)

*    MIT lecture [notes](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-042j-mathematics-for-computer-science-fall-2010/readings/MIT6_042JF10_chap04.pdf) on proofs